In [1]:
A_DIR = '/media/cdonahue/bulk1/datasets/pianoise/data/feats_melmagenta/ddc'
B_DIR = '/media/cdonahue/bulk1/datasets/pianoise/data/feats_melmagenta/maestro'

In [60]:
#A_FEATS_DIR = os.path.join(A_DIR, 'valid')
A_FEATS_DIR = '/media/cdonahue/bulk1/datasets/pianoise/data/feats_melmagenta/assorted'

import glob
import os
import random

import numpy as np

feats_fps = glob.glob(os.path.join(A_FEATS_DIR, '*.npy'))

feats_fp = random.choice(feats_fps)
print(feats_fp)

feats = np.load(feats_fp)
print(feats.shape)

/media/cdonahue/bulk1/datasets/pianoise/data/feats_melmagenta/assorted/undyne.npy
(469, 229, 1)


In [3]:
MODEL_DIR = '/home/cdonahue/cyclegan/pytorch-CycleGAN-and-pix2pix/checkpoints'
MODEL_NAME = '02_06_pianoise_default'
CKPT_FP = 'iter_135000'

from collections import namedtuple

from models.cycle_gan_model import CycleGANModel

opt = {
    'gpu_ids': [0],
    'isTrain': False,
    'checkpoints_dir': MODEL_DIR,
    'name': MODEL_NAME,
    'preprocess': 'none',
    'input_nc': 1,
    'output_nc': 1,
    'ngf': 64,
    'ndf': 64,
    'netG': 'resnet_9blocks',
    'netD': 'basic',
    'norm': 'instance',
    'no_dropout': True,
    'init_type': 'normal',
    'init_gain': 0.02,
}
opt = namedtuple('opt', opt.keys())(*opt.values())

model = CycleGANModel(opt)
model.load_networks(CKPT_FP)
model.eval()
device = model.device

initialize network with normal
initialize network with normal
loading the model from /home/cdonahue/cyclegan/pytorch-CycleGAN-and-pix2pix/checkpoints/02_06_pianoise_default/iter_135000_net_G_A.pth
loading the model from /home/cdonahue/cyclegan/pytorch-CycleGAN-and-pix2pix/checkpoints/02_06_pianoise_default/iter_135000_net_G_B.pth


In [4]:
BAND_FEATS = False

import os
import numpy as np

tag = 'band_' if BAND_FEATS else ''

a_mean = np.load(os.path.join(A_DIR, '{}mean.npy'.format(tag)))
a_std = np.load(os.path.join(A_DIR, '{}std.npy'.format(tag)))
b_mean = np.load(os.path.join(B_DIR, '{}mean.npy'.format(tag)))
b_std = np.load(os.path.join(B_DIR, '{}std.npy'.format(tag)))

print(a_mean.shape, a_std.shape, b_mean.shape, b_std.shape)

() () () ()


In [5]:
NUM_STD_DEVS = 5.

def _norm_feats(x, mean, std, num_std_devs=NUM_STD_DEVS):
    x = np.copy(x)
    x -= mean
    x /= std
    x /= num_std_devs
    return np.clip(x, -1., 1.)

def _unnorm_feats(x_norm, mean, std, num_std_devs=NUM_STD_DEVS):
    x = np.copy(x_norm)
    x *= num_std_devs
    x *= std
    x += mean
    return x

In [61]:
SLICE_LEN = 256
STRIDE = 32

import torch
from tqdm import tqdm

feats_norm = _norm_feats(feats, a_mean, a_std)

pad_amt = SLICE_LEN // 2
feats_norm_padded = np.pad(feats_norm, [[pad_amt, pad_amt], [0, 256-229], [0, 0]], 'reflect')

G_a_all = []
for i in tqdm(range(0, feats_norm.shape[0], STRIDE)):
    a = feats_norm_padded[i:i+SLICE_LEN]
    a = np.transpose(a, [2, 0, 1])
    assert a.shape == (1, 256, 256)
    
    a = torch.tensor(a).to(device)
    a = torch.unsqueeze(a, 0)
    G_a = model.netG_A(a)
    
    a = a.detach()
    G_a = G_a.detach()
    
    G_a_all.append(G_a[:, :, pad_amt:pad_amt+STRIDE])
G_a = torch.cat(G_a_all, dim=2)[0].cpu().numpy()
G_a = np.transpose(G_a, [1, 2, 0])
G_a = G_a[:feats.shape[0]]
G_a = G_a[:, :229]
G_a = _unnorm_feats(G_a, b_mean, b_std)
print(G_a.shape)

100%|██████████| 15/15 [00:00<00:00, 98.85it/s] 

(469, 229, 1)


In [62]:
OUT_DIR = './02_07_pianose_cyclegan_stride32'

if not os.path.isdir(OUT_DIR):
    os.makedirs(OUT_DIR)
out_fn = os.path.split(feats_fp)[1]
out_fp = os.path.join(OUT_DIR, out_fn)
np.save(out_fp, G_a)

In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt

def spec_preview(feats, nstart=1000, nlen=400):
    plt.figure(figsize=(16, 9))
    librosa.display.specshow(feats[nstart:nstart+nlen, :, 0].swapaxes(0, 1))

spec_preview(feats)
spec_preview(G_a)